                        Import CSV files to a postgres database 
Steps:
- import the CSV file into a pandas df
- clean the table name and remove all axtra symbols, spaces, capital letters
- clean the column headers and remove all axtra symbols, spaces, capital letters
- write the create table SQL statement
- import the data into the db

In [101]:
#import libraries
#conda install psycopg2
#pip install psycopg2

import os
import numpy as np 
import pandas as pd 
import psycopg2

In [102]:
!ls

Customer Contracts$.csv           customer_contracts.csv
Import_CSV_into_Postgres_DB.ipynb


In [103]:
df = pd.read_csv("Customer Contracts$.csv")
df.head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [104]:
#clean table names
#lower case letters
#remove all white spaces and $
#replace -,/,\\ with _

file = ("Customer Contracts$.csv")

clean_tbl_name = file.lower().replace(" ","_").replace("?","") \
                            .replace("-","_").replace(r"/","_").replace("\\","_").replace("$","") \
                            .replace(")","").replace(r"()","").replace("?","")

clean_tbl_name

'customer_contracts.csv'

In [105]:
# Save the cleaned DataFrame to a new CSV file
clean_tbl_name = "customer_contracts.csv"  # Use the cleaned name from your script
df.to_csv(clean_tbl_name, index=False)

In [106]:
#clean header names
#lower case letters
#remove all white spaces and $
#replace -,/,\\ with _

df.columns = [x.lower().replace(" ","_").replace("?","") \
                            .replace("-","_").replace(r"/","_").replace("\\","_").replace("$","") \
                            .replace(")","").replace(r"()","").replace("?","") for x in df.columns]

df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

create table customer_contracts
(
    customer_name varchar,
    start_date varchar,
    end_date varchar,
    contract_amount_m float,
    invoice_sent varchar,
    paid varchar
);

In [107]:
df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [108]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [109]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

In [114]:
#open a database connection

import psycopg2

conn_string = (
    "host=localhost "
    "dbname='csv_to_postgres' "
    "user='postgres' "
    "password='postgres'"
    "port=5432"
)

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('Opened local database successfully')

Opened local database successfully
